# ML Parsing #
Initial parsing of the grade submissions. It is meant only for research purposes and not for production environments.

_What it does?_
- Parses the intial csv files (form csv + grades uploaded in csv format)
- Shows basic info of the data
- Connects form submissions with their associated grades and cleans entry with no grades
- Parses the submission for the CSD Department
- Drops invalid values and fixes any other issues with the dataset
- Produces a final dataset with the form submissions and the grades. Each course found in the dataset is represented as a column. Only the valid courses are kept as found by the OPEN API of AUTH. The dataset contains 89 courses. 

*Some columns refer to the same course but they are with different codes (old/new study program)
- This is fixed, old courses are assigned to new course codes.

**ATTENTION!!: Some outputs have been deleted for privacy reasons. PLEASE DO NOT UPLOAD OUTPUTS WITH PERSONAL INFO** 

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
#Define files
form_csv = "Form_Responses_11_09.csv"
folder_with_grade_csvs = "../GradeParser/csv/"
# Json with the valid courses as got from OPEN AUTH API (https://ws-ext.it.auth.gr/)
valid_courses_json = "courses_ids_600000014.json"

## General Parsing ##

In [ ]:
#reading the form submissions (download csv file)
submissions_csv = pd.read_csv(form_csv)
display(submissions_csv)

In [ ]:
submissions_csv.info()

In [ ]:
#Create an extract column to store the paths of the csvs with the grades
csv = [None] * (submissions_csv.shape[0])
submissions_csv["CSV"] = pd.Series(csv).to_numpy()
# display(submissions_csv)
# print(submissions_csv['Filename'])

In [ ]:
#Find associated csv
for r,d,f in os.walk(folder_with_grade_csvs):
    for file in f:
        if ".csv" in file: # For each csv
            without_ext = file.replace("_results.csv","") #Remove extensions added by the parser\n",
            # Set the csv found \n",
            submissions_csv.loc[submissions_csv["Filename"].str.contains(without_ext,regex=False),"CSV"] = r + file
display(submissions_csv)

In [ ]:
# Nan CSV entries
display(submissions_csv[submissions_csv['CSV'].isnull()])
# Drop those entries with no csv associated
submissions_csv.dropna(subset=['CSV'],inplace = True)
display(submissions_csv)

## Parsing for Departments ##

In [ ]:
# Check unique departments
display(submissions_csv.groupby('Σχολή Φοίτησης')['Timestamp'].nunique())


# Get the CSD department
csd = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "Πληροφορική ΑΠΘ"]
# hmmy = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "ΗΜΜΥ ΑΠΘ"]


### CSD Department - Grade Statistics ###

In [ ]:
# Concat all grades (all the grade csvs together)
csd_csv_files = csd['CSV'].to_numpy()
li = []
# pd.read_csv(csd_csv_files[5],encoding='utf-8')
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
# coltypes = {"C_ID":str,"C_NAME":str,"TYPE":str,"EXAM_YEAR":str,"EXAM_PERIOD":str,"ECTS":np.int32,"DM":np.int32,"FACTOR":np.int32,"GRADE":np.float64}
for csv_file in csd_csv_files:
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    li.append(df)

csd_grades = pd.concat(li, axis=0, ignore_index=True)
display(csd_grades)

In [ ]:
# Find more information on the courses
unique_courses = csd_grades["C_ID"].unique()
print("Total unique courses:{}".format(len(unique_courses)))
csd_grades['C_ID'] = csd_grades['C_ID'].str.replace(' ','',regex=True)
csd_grades["C_ID"].value_counts()

In [ ]:
# Fixing Issues
def fix_issues(df):
    
    df['C_ID'] = df['C_ID'].str.replace(' ','',regex=True) # Removing blank cases
    
    df['C_NAME'] = df['C_NAME'].str.replace('*','',regex=True) # Removing stars
    df['C_NAME'] = df['C_NAME'].str.strip() # Removing leading and trailing spaces

    # Fixing issue with parser
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΤΙΚΕΙΜΕΝΟΣΤΡΕΦΗΣ','ΑΝΤΙΚΕΙΜΕΝΟΣΤΡΕΦΗΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΣ',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ','ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ ΑΥΤΟΜΑΤΩΝ',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΤΕΧΝΙΚΕΣ ΔΟΜΗΣΗΣ','ΤΕΧΝΙΚΕΣ ΔΟΜΗΣΗΣ ΔΕΔΟΜΕΝΩΝ',regex=False)
    
    # Grouping old courses with new ones with the same name - CHANGED TO MANUALLY
    unique_courses = df['C_NAME'].unique()
    for course in unique_courses:
        course_grades = df[df['C_NAME'] == course]
        if course_grades['C_ID'].nunique() > 1:
            course_codes = course_grades['C_ID'].value_counts().index.values
            for i in range(1,len(course_codes)):
                df['C_ID'] = df['C_ID'].replace(course_codes[i],course_codes[0],regex=False)
                
                
    ## Manual Work ## 
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΑΓΝΩΡΙΣΗ ΠΡΟΤΥΠΩΝ','ΑΝΑΓΝΩΡΙΣΗ ΠΡΟΤΥΠΩΝ -ΣΤΑΤΙΣΤΙΚΗ ΜΑΘΗΣΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('NDM-06-03','NDM-06-04',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΩΤΕΡΑ ΜΑΘΗΜΑΤΙΚΑ Ι','ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ Ι',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-01','NCO-01-01',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΑΝΩΤΕΡΑ ΜΑΘΗΜΑΤΙΚΑ ΙΙ','ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ ΙΙ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-01','NCO-02-01',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΕΙΣΑΓΩΓΗ ΣΤΟΥΣ ΥΠΟΛΟΓΙΣΤΕΣ','ΕΙΣΑΓΩΓΗ ΣΤΗΝ ΠΛΗΡΟΦΟΡΙΚΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-02','NCO-01-02',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΘΕΩΡΙΑ ΓΛΩΣΣΩΝ ΚΑΙ ΑΥΤΟΜΑΤΩΝ','ΘΕΩΡΙΑ ΥΠΟΛΟΓΙΣΜΟΥ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-04','NCO-02-05',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΝΕΥΡΩΝΙΚΑ ΔΙΚΤΥΑ','ΝΕΥΡΩΝΙΚΑ ΔΙΚΤΥΑ - ΒΑΘΙΑ ΜΑΘΗΣΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('NDM-07-03','NDM-07-05',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΠΙΘΑΝΟΤΗΤΕΣ KAI ΣΤΑΤΙΣΤΙΚΗ','ΠΙΘΑΝΟΤΗΤΕΣ & ΣΤΑΤΙΣΤΙΚΗ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-02','NCO-02-02',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΣΧΕΔΙΑΣΗ ΑΛΓΟΡΙΘΜΩΝ','ΑΛΓΟΡΙΘΜΟΙ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-05','NCO-04-03',regex=False)
    df['C_NAME'] = df['C_NAME'].replace('ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΚΑΙ ΜΕΤΑΓΛΩΤΤΙΣΤΕΣ','ΣΧΕΔΙΑΣΗ ΓΛΩΣΣΩΝ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ ',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-03','NCO-04-01',regex=False)
    
    # More manual
    df['C_ID'] = df['C_ID'].replace('CO-01-05','NCO-01-05',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-06','NCO-01-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-01-03','NCO-01-03',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-02-05','NCO-02-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-02','NCO-04-05',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-03-03','NCO-03-04',regex=False)
    df['C_ID'] = df['C_ID'].replace('CO-04-04','NCO-04-02',regex=False)
    
    return df

In [ ]:
# Printing the full dataframe
def print_full(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [ ]:
# Checking more info
unique_names = csd_grades.groupby('C_NAME')['C_ID'].value_counts()
print_full(unique_names)
print("Total:" + str(csd_grades['C_NAME'].nunique()))


In [ ]:
csd_grades = fix_issues(csd_grades)

In [ ]:
unique_names = csd_grades.groupby('C_NAME')['C_ID'].value_counts()
print_full(unique_names)
print("Total:" + str(csd_grades['C_NAME'].nunique()))

In [ ]:
# Get's the real courses offered as got from the AUTH Open Data API (https://ws-ext.it.auth.gr/)
def get_valid_courses():
    with open(valid_courses_json) as json_file:
        data = json.load(json_file)
        df = pd.DataFrame(data['courses'])
    json_file.close()
    return df['coursecode'].values

In [ ]:
import numpy as np
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
unique_courses = csd_grades["C_ID"].unique()
course_dictionary = dict()
# Create a dictionary for each course, each row represents a student
for course in get_valid_courses():
    course_dictionary[course] = [-1] * len(csd_csv_files) 
for i in range(0,len(csd_csv_files)):
    csv_file = csd_csv_files[i]
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    df = fix_issues(df)
    df_courses = df["C_ID"].to_numpy()
    for course_id in df_courses:
        if course_id in course_dictionary:
            course_dictionary[course_id][i] =  df[df["C_ID"] == course_id]["GRADE"].to_numpy()[0]
        else:
            print("Not valid course:" + course_id + '-' + csd_grades[csd_grades["C_ID"] == course_id]['C_NAME'].values)
csd_courses_df = pd.DataFrame.from_dict(course_dictionary)
display(csd_courses_df)

In [ ]:
# Fill nan values with -1
csd_courses_df.fillna(-1,inplace = True)
# Replace blank values with -1
csd_courses_df.replace(r'^\s*$', -1, regex=True,inplace=True)
# Replace ΕΠΙΤ with 10
csd_courses_df.replace(' ΕΠΙΤ',10,inplace=True)
display(csd_courses_df)

In [ ]:
# Change to type flaot
csd_courses_df = csd_courses_df.astype(np.float64)

In [ ]:
csd_courses_df.describe()

#### Concatenate CSD Student + Grades ####

In [ ]:
# Remove unnecessary fields for final dataset
csd_final = csd.drop(['Timestamp', 'Σχολή Φοίτησης',
       'Υποβολή αναλυτικής βαθμολογίας',
       'Συναινώ στη χρήση των στοιχείων μου από την ομάδα του Grade++',
       'Αν θες να λάβεις πρώτος πρόσβαση στην εφαρμογή, συμπλήρωσε το email σου:',
       'Filename', 'CSV'],axis=1)
csd_final.reset_index(drop=True,inplace=True)
display(csd_final)

In [ ]:
# Concat with grades
ldf = [csd_final, csd_courses_df]
csd_final = pd.concat(ldf,axis=1)
display(csd_final)

In [ ]:
# Write to file
csd_final.to_csv("csd.csv",index=False)